In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize, basinhopping
from itertools import product
from numpy.linalg import cholesky
from tqdm import tqdm

import chart_studio.plotly as py
import plotly.graph_objects as go

import funciones_optim
from importlib import reload
reload(funciones_optim)

from funciones_optim import port_ret,sumauno,var,r_gen,min_vol

In [2]:
data = pd.read_csv('series_tiempo.csv',index_col='FECHA')
data.index= pd.to_datetime(data.index,format='%d/%m/%Y')
data.sort_index(inplace=True)
data = data.dropna(axis=0).drop(['CASH'],axis=1)

vol = data.mul(np.sqrt(12)).std()
cor = data.corr()
cov = cor*np.outer(vol,vol)
ret = pd.Series({k:v for k,v in zip(vol.index,[8.8,9,13])})

In [13]:
perfiles = 10  #Perfiles
num_simu = 100  #numero de simulaciones
w = np.array([ 1/len(vol) for i in range(len(vol))])
bounds = [(0,1) for i in range(len(vol))]
minv = min_vol(bounds,cov,w)
maxv = np.max(vol)
target_vol = np.linspace(minv,maxv,num=perfiles)
R = r_gen(num_simu,ret,cor,vol)

opt_ports = {}
for tv in tqdm(target_vol):
    ws_ =[]
    for ri in R:
        constraints = [{'type':'eq','fun':var,'args':(cov,tv)},{'type':'eq','fun':sumauno}]
        result = minimize(port_ret,w,args=(ri),bounds=bounds,constraints=constraints)
        if result.success:
            ws_.append(result.x)
        
    opt_ports[np.round(tv*100,2)] = pd.Series(ws_).mean(axis=0).round(4)
df = pd.DataFrame(opt_ports,index=ret.index).transpose()
df['Return'] = df.apply(port_ret,args=[ret],axis=1).mul(-1).round(3)
df.index.name = 'Volatility'

100%|██████████| 10/10 [00:06<00:00,  1.44it/s]


In [16]:
traces=[]
traces.append(go.Scatter(x=df.index,y=df.Return,mode='lines+markers',line={'width':1,'color':'black'},marker={'size':5,'color':'blue'}))
layout = go.Layout(title={'text':'Frontera Eficiente','font':{'size':15}},width=500,height=400,
                   margin={'t':50,'l':20,'r':20,'b':20},template='simple_white',
                   xaxis={'title':{'text':'xVolatility','font':{'size':12}}},
                   yaxis={'title':{'text':'xReturn','font':{'size':12}}})
fig = go.Figure(traces,layout)
fig.show()

